Assignment: Flowers Recognition <br>
Dataset Description:<br>

This dataset contains 4242 images of flowers.<br>
The data collection is based on the data flicr, google images, yandex images.<br>
You can use this datastet to recognize plants from the photo.<br>

Attribute Information:<br>
The pictures are divided into five classes: chamomile, tulip, rose, sunflower, dandelion.<br>
For each class there are about 800 photos. Photos are not high resolution, about 320x240 pixels. <br>
<b>Also explore how to resize images in tensorflow and then resize all the images to a same size. </b> <br>
This is a Multiclass Classification Problem.<br>




WORKFLOW : <br>
Load Data <br>
Split into 60 and 40 ratio.<br>
Encode labels.<br>
Create Model<br>
Compilation Step (Note : Its a Multiclass Classification problem , select loss , metrics according to it)<br>
Train the Model.<br>
If the model gets overfit tune your model by changing the units , No. of layers , epochs , add dropout layer or add Regularizer according to the need .<br>
Prediction should be > 85%<br>
Evaluation Step<br>
Prediction<br>




Data : <br>
https://drive.google.com/file/d/1-OX6wn5gA-bJpjPNfSyaYQLz-A-AB_uj/view?usp=sharing

In [1]:
import cv2
import glob
import numpy as np 
import pandas as pd 
import tensorflow as tf 
from tensorflow.keras import layers, models, optimizers, Sequential

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
cd /content/drive/MyDrive/Assignment3_DL/flowers

/content/drive/MyDrive/Assignment3_DL/flowers


In [5]:
# Defining function

flowers = ['./daisy', './dandelion', './rose', './sunflower', './tulip']
length = []

def read_image(directory):
    for img in glob.glob(directory+'/*.jpg'):
        image = cv2.imread(img)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        resized_img = cv2.resize(image/255.0, (100,100))
        yield resized_img


In [6]:
for x in flowers:
    print(x)
    y = np.array(list(read_image(x)))
    if x == './daisy':
        data = y  
    else:
        data = np.vstack((data, y))
    length.append(len(y))

data.shape, length

./daisy
./dandelion
./rose
./sunflower
./tulip


((4324, 100, 100), [769, 1052, 785, 734, 984])

In [7]:
data = data.reshape((4324,100*100))
data = data.astype('float32')

In [8]:
i = 1
targets = np.ones((length[0], 1))*0
for x in length[1:]:
    targets = np.vstack((targets, np.ones((x , 1)) * i))
    i += 1

targets

array([[0.],
       [0.],
       [0.],
       ...,
       [4.],
       [4.],
       [4.]])

In [9]:
targets.shape

(4324, 1)

In [10]:
(targets == 0).sum(), (targets == 1).sum(), (targets == 2).sum(), (targets == 3).sum(), (targets == 4).sum()

(769, 1052, 785, 734, 984)

In [15]:
# train_images = train_data.reshape((2607,100*100))
# train_images = train_images.astype('float32')
# test_images = test_data.reshape((1717,100*100))
# test_images = test_images.astype('float32')

# train_images, test_images

In [11]:
np.random.seed(111)
mask = np.random.rand(len(data)) < 0.6
train_data = data[mask]
test_data = data[~mask]
train_targets = targets[mask]
test_targets = targets[~mask]

In [12]:
train_data.shape, test_data.shape, train_targets.shape, test_targets.shape

((2607, 10000), (1717, 10000), (2607, 1), (1717, 1))

In [13]:
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(100*100,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
model.fit(train_data, train_targets, epochs = 5, batch_size = 100)

Epoch 1/5
27/27 [==============================] - 1s 13ms/step - loss: 0.0000e+00 - accuracy: 0.2405
Epoch 2/5
27/27 [==============================] - 0s 14ms/step - loss: 0.0000e+00 - accuracy: 0.2370
Epoch 3/5
27/27 [==============================] - 0s 14ms/step - loss: 0.0000e+00 - accuracy: 0.2387
Epoch 4/5
27/27 [==============================] - 0s 14ms/step - loss: 0.0000e+00 - accuracy: 0.2344
Epoch 5/5
27/27 [==============================] - 0s 14ms/step - loss: 0.0000e+00 - accuracy: 0.2427


In [17]:
model.evaluate(test_data, test_targets)

54/54 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 0.2487


[0.0, 0.24868957698345184]

In [19]:
with tf.device('/device:GPU:0'):
  result = model.fit(train_data, train_targets, epochs = 5, batch_size = 100)

Epoch 1/5
27/27 [==============================] - 0s 14ms/step - loss: 0.0000e+00 - accuracy: 0.2397
Epoch 2/5
27/27 [==============================] - 0s 13ms/step - loss: 0.0000e+00 - accuracy: 0.2397
Epoch 3/5
27/27 [==============================] - 0s 14ms/step - loss: 0.0000e+00 - accuracy: 0.2397
Epoch 4/5
27/27 [==============================] - 0s 14ms/step - loss: 0.0000e+00 - accuracy: 0.2397
Epoch 5/5
27/27 [==============================] - 0s 14ms/step - loss: 0.0000e+00 - accuracy: 0.2397


In [20]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4321413882490094826
]
